In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

import os
import sys

curr_dir = os.getcwd()
print('current direcotry is ', curr_dir)
if curr_dir not in sys.path:
    sys.path.append(curr_dir)
src_dir = os.path.join(curr_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

#
pdb_dir = '/home/yuan/data/pdb/rsync'
data_dir = './data/'
outputs_dir = '/home/yuan/output/pdb_abag'

current direcotry is  /home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex


## PDB

### SAbDab

In [172]:
from src.dir import Dir

sabdab_summary, n = [], 0
file_iter = Dir('/home/yuan/data/sabdab/summary').recrusive_files()
for path in file_iter:
    if path.endswith('.tsv'):
        df = pd.read_csv(path, sep='\t')
        sabdab_summary.append(df)
sabdab_summary = pd.concat(sabdab_summary, axis=0)
sabdab_summary['pdb'] = sabdab_summary['pdb'].str.upper()
sabdab_summary = sabdab_summary[~sabdab_summary['antigen_chain'].isna()]
# sabdab_summary[(sabdab_summary['Hchain'].isna()) & (sabdab_summary['Lchain'].isna())]
print(sabdab_summary.shape)
sabdab_summary.head()

(7418, 30)


,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,3WHX,A,B,0,B,Hapten,XPG,"7-[(1R,3R)-3-hydroxy-2-[(1E,3S)-3-hydroxyoct-1...",IMMUNE SYSTEM,09/03/13,...,False,False,IGHV1,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
0,1NSN,H,L,0,S,protein,NaN,staphylococcal nuclease,COMPLEX (IMMUNOGLOBULIN/HYDROLASE),06/06/95,...,False,False,IGHV3,IGKV3,Kappa,1.000000e-10,-13.63,Other,NaN,1704035.0
0,1R3L,B,A,0,C,protein,NaN,voltage-gated potassium channel,MEMBRANE PROTEIN,10/02/03,...,False,False,IGHV1,IGKV5,Kappa,NaN,NaN,NaN,NaN,NaN
0,6J7W,A,NaN,0,C,peptide,NaN,tumor necrosis factor receptor superfamily mem...,IMMUNE SYSTEM,01/18/19,...,False,True,IGHV3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6J7W,B,NaN,0,D,peptide,NaN,tumor necrosis factor receptor superfamily mem...,IMMUNE SYSTEM,01/18/19,...,False,True,IGHV3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
cols = ['pdb_id', 'model',  'chain_id']

df1 = sabdab_summary[['pdb', 'model', 'Hchain']]
df1.columns = cols
df1 = df1[~df1['chain_id'].isna()]
df1 = df1.assign(chain_type = 'heavy')

df2 = sabdab_summary[['pdb','model','Lchain']]
df2.columns = cols
df2 = df2[~df2['chain_id'].isna()]
df2 = df2.assign(chain_type = 'light')


df3 = sabdab_summary[['pdb','model','antigen_chain']]
df3.columns = cols
df3 = df3.assign(chain_type = 'receptor')

# concatenate
sabdab = pd.concat([df1, df2, df3])
sabdab = sabdab.sort_values(cols)
print(sabdab.shape)
sabdab.head()

(20964, 4)


,pdb_id,model,chain_id,chain_type
0,1A14,0,H,heavy
0,1A14,0,L,light
0,1A14,0,N,receptor
0,1A2Y,0,A,light
0,1A2Y,0,B,heavy


In [182]:
sabdab = sabdab.drop_duplicates()
print(sabdab.shape, sabdab.dtypes)
print(sabdab.head())
sabdab_pdb = sabdab['pdb_id'].unique()

(20555, 4) pdb_id        object
model          int64
chain_id      object
chain_type    object
dtype: object
  pdb_id  model chain_id chain_type
0   1A14      0        H      heavy
0   1A14      0        L      light
0   1A14      0        N   receptor
0   1A2Y      0        A      light
0   1A2Y      0        B      heavy


### this study

In [183]:
from src.collect import Collect

abag_chains = []
meta_iter = Collect('/home/yuan/output/pdb_abag').meta()
for _data in meta_iter:
    abag_chains.extend(_data)
abag_chains = pd.concat(abag_chains, axis=0)
abag_chains = abag_chains[['pdb_id', 'model',  'chain_id', 'chain_type']]
abag_chains = abag_chains.drop_duplicates()
print(abag_chains.shape, abag_chains.dtypes)
abag_chains.head()

abag_pdb = abag_chains['pdb_id'].unique()

(50107, 4) pdb_id        object
model          int64
chain_id      object
chain_type    object
dtype: object


In [187]:
set(sabdab_pdb).difference(abag_pdb)

{'4HEP',
 '6HHD',
 '1YED',
 '5NHR',
 '6IDK',
 '6GS4',
 '6OS1',
 '6NBF',
 '1IGJ',
 '4BFB',
 '7MJI',
 '1NFD',
 '7ME7',
 '5GS2',
 '3HZM',
 '6XUN',
 '3T0X',
 '5IWL',
 '5G64',
 '4W6W',
 '6QGX',
 '6SS4',
 '6O3U',
 '6YZ5',
 '3J42',
 '5E5M',
 '6M1I',
 '6HHU',
 '6VCB',
 '6IBB',
 '6E3Y',
 '6N51',
 '5MP3',
 '4X7E',
 '4UIN',
 '7A4D',
 '2DQJ',
 '4KFZ',
 '5T3X',
 '35C8',
 '1FL3',
 '1E4X',
 '6HUJ',
 '3FO9',
 '5NBL',
 '2ADF',
 '6B3D',
 '6EJG',
 '4PD4',
 '4NJA',
 '7DWT',
 '3CFK',
 '3CFD',
 '6JU0',
 '5TJW',
 '3JBA',
 '6QV0',
 '7DUQ',
 '1INE',
 '4MQS',
 '7LMN',
 '5DT1',
 '6Y49',
 '6B73',
 '7RNN',
 '4WEN',
 '4FQR',
 '4C57',
 '1UA6',
 '2R2E',
 '1MCD',
 '6LPB',
 '6H72',
 '5T4Z',
 '4NC2',
 '6RTW',
 '1MFA',
 '7DHR',
 '4OB5',
 '6QPG',
 '7C2E',
 '7LVW',
 '6GS1',
 '2H1P',
 '4D3C',
 '4NBX',
 '7LDJ',
 '7K65',
 '7P16',
 '6T9E',
 '6DBF',
 '7CKW',
 '7N65',
 '6Y1N',
 '5DFW',
 '6JB5',
 '6ZHD',
 '6UG7',
 '5ACL',
 '3EYF',
 '4I1N',
 '2R9H',
 '7OAO',
 '2AJV',
 '6BT3',
 '6QTL',
 '6E5P',
 '3J8V',
 '6DB9',
 '3C6S',
 '6OS0',
 

In [179]:
cols = ['pdb_id', 'chain_id']
abag_sabdab1 = pd.merge(abag_chains, sabdab, on=cols, how='left')
print(abag_sabdab1.shape)
abag_sabdab2 = pd.merge(abag_chains, sabdab, on=cols, how='inner')
print(abag_sabdab2.shape)
abag_sabdab3 = pd.merge(abag_chains, sabdab, on=cols, how='right')
print(abag_sabdab3.shape)

(50300, 6)
(14148, 6)
(20735, 6)


In [180]:
abag_sabdab3

,pdb_id,model_x,chain_id,chain_type_x,model_y,chain_type_y
0,1A14,0.0,H,heavy,0,heavy
1,1A14,0.0,L,light,0,light
2,1A14,0.0,N,receptor,0,receptor
3,1A2Y,0.0,A,light,0,light
4,1A2Y,0.0,B,heavy,0,heavy
...,...,...,...,...,...,...
20730,NaN,NaN,L,NaN,0,receptor
20731,NaN,NaN,N,NaN,0,heavy
20732,NaN,NaN,N,NaN,0,receptor
20733,NaN,NaN,P,NaN,0,receptor


In [128]:
sabdab_chains[sabdab_chains['pdb_id']=='9FGS']

,pdb_id,model,chain_id,chain_type
